In [1]:
import os
import sys
import logging
import pandas as pd
from typing import Dict

In [2]:
sys.path.append(os.path.dirname(os.getcwd()))
from configs import BASE_MAPPING_PATH, QANDA_FILE_READER_PATH, PIPELINE_FIELD_MAP
from client import (
    OsMlClientWrapper,
    get_client,
)
from data_process import QAndAFileReader
from main import get_ml_model, load_category
from mapping import get_base_mapping, mapping_update
from ml_models import get_remote_connector_configs, MlModel

In [3]:
def load_dataset_lexical(
    client: OsMlClientWrapper,
    pqa_reader: QAndAFileReader,
    config: Dict[str, str],
    delete_existing: bool,
    index_name: str,
):
    if delete_existing:
        logging.info(f"Deleting existing index {index_name}")
        client.delete_then_create_index(
            index_name=config["index_name"], settings=config["index_settings"]
        )

    logging.info("Setting up without KNN")
    client.setup_without_kNN(
        index_name=config["index_name"],
        index_settings=config["index_settings"],
    )

    for category in config["categories"]:
        load_category(
            client=client.os_client,
            pqa_reader=pqa_reader,
            category=category,
            config=config,
        )

In [4]:
def load_dataset_dense(
    client: OsMlClientWrapper,
    ml_model: MlModel,
    pqa_reader: QAndAFileReader,
    config: Dict[str, str],
    delete_existing: bool,
    index_name: str,
    pipeline_name: str,
):
    if delete_existing:
        logging.info(f"Deleting existing index {index_name}")
        client.delete_then_create_index(
            index_name=config["index_name"], settings=config["index_settings"]
        )

    logging.info("Setting up for KNN")
    client.setup_for_kNN(
        ml_model=ml_model,
        index_name=config["index_name"],
        pipeline_name=pipeline_name,
        index_settings=config["index_settings"],
        pipeline_field_map=config["pipeline_field_map"],
        delete_existing=delete_existing,
        embedding_type=config["embedding_type"],
    )

    for category in config["categories"]:
        load_category(
            client=client.os_client,
            pqa_reader=pqa_reader,
            category=category,
            config=config,
        )

In [5]:
def create_dense_index_settings(base_mapping_path, index_config):
    settings = get_base_mapping(base_mapping_path)
    pipeline_name = index_config["pipeline_name"]
    model_dimension = index_config["model_dimensions"]
    knn_settings = {
        "settings": {"index": {"knn": True}, "default_pipeline": pipeline_name},
        "mappings": {
            "properties": {
                "chunk": {"type": "text", "index": False},
                "chunk_embedding": {
                    "type": "knn_vector",
                    "dimension": model_dimension,
                },
            }
        },
    }
    mapping_update(settings, knn_settings)
    return settings

In [6]:
host_type = "aos"
number_of_docs = 500
dataset_path = QANDA_FILE_READER_PATH
categories = ["sheet and pillowcase sets"]

client = OsMlClientWrapper(get_client(host_type))

pqa_reader = QAndAFileReader(directory=dataset_path, max_number_of_docs=number_of_docs)

2025-04-10:08:23:55,902 INFO     [base.py:258] GET https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/ [status:200 request:0.052s]
2025-04-10:08:23:55,922 INFO     [base.py:258] GET https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/_plugins/_ml/model_groups/_search [status:200 request:0.020s]


In [7]:
lexical_index_name = "lexical_search"

lexical_index_config = {
    "categories": categories,
    "index_name": lexical_index_name,
    "index_settings": get_base_mapping(BASE_MAPPING_PATH),
}

load_dataset_lexical(
    client,
    pqa_reader,
    lexical_index_config,
    delete_existing=True,
    index_name=lexical_index_name,
)

2025-04-10:08:23:55,945 INFO     [2292998341.py:9] Deleting existing index lexical_search
2025-04-10:08:23:55,958 INFO     [base.py:258] HEAD https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/lexical_search [status:200 request:0.012s]
2025-04-10:08:23:55,958 INFO     [os_ml_client_wrapper.py:115] Deleting index lexical_search
2025-04-10:08:23:56,249 INFO     [base.py:258] DELETE https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/lexical_search [status:200 request:0.291s]
2025-04-10:08:23:56,674 INFO     [base.py:258] PUT https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/lexical_search [status:200 request:0.424s]
2025-04-10:08:23:56,675 INFO     [os_ml_client_wrapper.py:100] idempotent_create_index response: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'lexical_search'}
2025-04-10:08:23:56,675 INFO     [2292998341.py:14] Setting u

In [8]:
embedding_type = "dense"
dense_model_type = "sagemaker"
dense_index_name = "dense_search"
dense_model_name = "Sagemaker Dense Model"
dense_pipeline_name = "dense_embedding_pipeline"

dense_index_config = {
    "with_knn": True,
    "categories": categories,
    "index_name": dense_index_name,
    "embedding_type": embedding_type,
    "pipeline_name": dense_pipeline_name,
    "pipeline_field_map": PIPELINE_FIELD_MAP,
}

dense_model_config = get_remote_connector_configs(
    host_type=host_type, connector_type=dense_model_type
)
dense_model_config["model_name"] = dense_model_name
dense_model_config["embedding_type"] = embedding_type

dense_index_config.update(dense_model_config)

dense_model = get_ml_model(
    client=client,
    host_type=host_type,
    model_type=dense_model_type,
    model_config=dense_model_config,
)

dense_index_config["index_settings"] = create_dense_index_settings(
    base_mapping_path=BASE_MAPPING_PATH,
    index_config=dense_index_config,
)

load_dataset_dense(
    client,
    dense_model,
    pqa_reader,
    dense_index_config,
    delete_existing=True,
    index_name=dense_index_name,
    pipeline_name=dense_pipeline_name,
)

2025-04-10:08:23:58,983 INFO     [credentials.py:1278] Found credentials in shared credentials file: ~/.aws/credentials
2025-04-10:08:23:59,184 INFO     [base.py:258] POST https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/_plugins/_ml/connectors/_search [status:200 request:0.006s]
2025-04-10:08:23:59,191 INFO     [base.py:258] POST https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/_plugins/_ml/connectors/_search [status:200 request:0.006s]
2025-04-10:08:23:59,191 INFO     [ml_connector.py:38] Connector id 0gJSHpYBdIibWvQqjx9b
2025-04-10:08:23:59,197 INFO     [base.py:258] POST https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/_plugins/_ml/models/_search [status:200 request:0.005s]
2025-04-10:08:23:59,203 INFO     [base.py:258] POST https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/_plugins/_ml/mod

In [9]:
query_text = "The most beautiful pillow"
search_size = 50

lexical_search_query = {
    "size": search_size,
    "_source": {"include": "chunk"},
    "query": {"match": {"chunk": query_text}},
}
dense_search_query = {
    "size": search_size,
    "_source": {"include": "chunk"},
    "query": {
        "neural": {
            "chunk_embedding": {
                "k": search_size,
                "query_text": query_text,
                "model_id": dense_model.model_id(),
            }
        }
    },
}

In [10]:
lexical_search_results = client.os_client.search(
    index=lexical_index_name, body=lexical_search_query
)

dense_search_results = client.os_client.search(
    index=dense_index_name, body=dense_search_query
)

lexical_hits = lexical_search_results["hits"]["hits"]
lexical_hits = [lexical_hit["_source"]["chunk"] for lexical_hit in lexical_hits]
lexical_hits = list(set(lexical_hits))

dense_hits = dense_search_results["hits"]["hits"]
dense_hits = [dense_hit["_source"]["chunk"] for dense_hit in dense_hits]
dense_hits = list(set(dense_hits))

2025-04-10:08:24:05,215 INFO     [base.py:258] POST https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/lexical_search/_search [status:200 request:0.015s]
2025-04-10:08:24:05,261 INFO     [base.py:258] POST https://search-opensearch-ml-quickstart-bpt2cjvxdhn7xan6ogr7pdd53m.us-west-2.es.amazonaws.com:443/dense_search/_search [status:200 request:0.046s]


In [11]:
top10_dense_hits, top10_lexical_hits = dense_hits[:10], lexical_hits[:10]

df = pd.DataFrame(
    {
        "top 10 dense search results": top10_dense_hits,
        "top 10 lexical search results": top10_lexical_hits,
    }
)

In [12]:
from IPython.display import display, HTML

# Create a title and style the DataFrame
styled_df = (
    df.style.set_caption(f"Search Results from query: {query_text}")
    .set_table_styles(
        [
            {
                "selector": "caption",
                "props": [
                    ("text-align", "center"),
                    ("font-size", "20px"),
                    ("font-weight", "bold"),
                    ("color", "#333333"),
                    ("background-color", "#f0f0f0"),
                    ("padding", "10px"),
                ],
            },
            {
                "selector": "th",
                "props": [
                    ("font-size", "16px"),
                    ("text-align", "center"),
                    ("background-color", "#4472C4"),
                    ("color", "white"),
                    ("font-weight", "bold"),
                    ("padding", "10px"),
                ],
            },
            {
                "selector": "td",
                "props": [
                    ("font-size", "14px"),
                    ("padding", "8px"),
                    ("border", "1px solid #ddd"),
                ],
            },
            {
                "selector": "tr:nth-of-type(odd)",
                "props": [("background-color", "#f9f9f9")],
            },
        ]
    )
    .format_index(str.upper, axis=1)
    .relabel_index([str(num) for num in range(1, 11)], axis=0)
    .set_properties(**{"max-width": "800px", "white-space": "pre-wrap"})
)

# Display the styled DataFrame with increased size
display(HTML("<style>.rendered_html table {font-size: 16px; width: 100%;}</style>"))
display(styled_df)

,TOP 10 DENSE SEARCH RESULTS,TOP 10 LEXICAL SEARCH RESULTS
1,"Wrap yourself in the Softness of the luxurious 100% cotton CRISPY PERCALE SHEETS like those found in World Class Hotels. These fine luxury bed linens are crafted from long staple cotton with Traditional, Original, Crisp, Breathable Percale Weave for a real good and healthy sleep. Comfort, quality and opulence set our luxury bedding in a class above the rest. Elegant yet durable, their softness is enhanced with each washing. If you have never experienced the luxury and elegance of high quality Crispy Percale sheets.... try it! There is a difference ! Royal Hotel Abripedic Crispy Percale Sheets, 300-Thread-Count, 4PC Solid Sheet Set, 100% Cotton, 22 Inch Super Deep Pocket, California-King, Gray","Wake Up to a Brighter TomorrowThe search is over! Say goodbye to restless, sweaty nights and usher in a new era of peaceful, refreshing sleep. Our luxurious fitted sheets are tightly woven and double brushed on both sides, contributing to a silky soft feel and ensuring that our 100% microfiber sheets are softer and more durable than Egyptian cotton sheets, organic cotton sheets or 1800 thread count sheets! Extra Deep Pocket Fitted SheetsThese are the best sheets for extra tall & large mattresses, and great for mattresses with a 3-inch or 4-inch topper, double pillow top, memory foam topper or thick mattress pad. A stretchable elastic all around the entire sheet ensures a tight, secure fit that won’t ride up or move around in middle of the night, staying smooth, snug and wrinkle-free. Pet Friendly No worries about pet nails and claws! Our bedding is less likely to tear or snag, even when you share your bed with your pets. So go ahead, build a pillow fort and snuggle inside with your best friend. Lightweight, breathable and cool to the touch, our dreamy, soft bedding will create an unforgettable sleeping experience! Nestl Into HappinessWe created Nestl to bring you soft, beautiful bedding at fair prices. As pioneers at the forefront of the bedding industry, we are proud to present you with affordable, high-quality bedding, crafted from premium microfiber material and made to last. Nestle between our silky soft bedding for a sleep so sublime, you’ll wonder if you’re dreaming! Extra Deep Pocket Fitted Sheet - Deep King Fitted Sheet Set Super Deep Pocket Sheet - Ultra Deep Fitted Sheet - for Extra Deep Pocket Mattress - King Size Sheets - 78 x 80 inches - Teal"
2,"Linenwalas Luxury Bamboo Sheets - 4-Piece Bed Sheet Set - Softest bedsheets and Pillow Cases - Sateen finish - Queen - Bahamas Blueluxurious 100% bamboo sheets like those found Hotel Life at homes. You wont be able to go back to cotton sheets after trying these Plain 100% bamboo sheets. These fine 300 TC luxury bed linens are 60% more absorbent than cotton. Comfort,quality and opulence set our luxury bedding in a class above the rest.Machine wash in cold water.Delicate cycle with mild detergent. ABOUT LINENWALAS:- Linenwalas is a premium brand that manufactures and supplies high quality Luxury and Hospitality products. Linenwalas is backed and promoted by IIT Alumni which ensures highest standards for our products.The brand offers a wide range of Home Furnishing products including Pillow and Covers,Curtains,Blankets and Quilts,Bed and Bath linen and many others. The brand is committed to provide Home Furnishing solutions that keep pace with the changing trends of global fashion. JUST TRY OUR PRODUCT and we guarantee that you won't be disappointed. You will be amaSized by the customer service we provide after you purchase our product. Best Quality Guaranteed. Our Products Are Tested To Maintain The Highest Standards Fastest Returns Guaranteed. We insts on the best quality and price for every ngle product,meanwhile we promised our best service for every ngle customer.Allows natural light to flow through the room. Color may not appear as exactly as in real life due to variations between the computer monitors. LINENWALAS Org